http://tflearn.org/tutorials/quickstart.html



In [1]:
import numpy as np
import tflearn


# Download the Titanic dataset
from tflearn.datasets import titanic
titanic.download_dataset('titanic_dataset.csv')

# Load CSV file, indicate that the first column represents labels
from tflearn.data_utils import load_csv
data, labels = load_csv('titanic_dataset.csv', target_column=0,
                        categorical_labels=True, n_classes=2)
# Preprocessing function
def preprocess(data, columns_to_ignore):
    # Sort by descending id and delete columns
    for id in sorted(columns_to_ignore, reverse=True):
        [r.pop(id) for r in data]
    for i in range(len(data)):
      # Converting 'sex' field to float (id is 1 after removing labels column)
      data[i][1] = 1. if data[i][1] == 'female' else 0.
    return np.array(data, dtype=np.float32)

# Ignore 'name' and 'ticket' columns (id 1 & 6 of data array)
to_ignore=[1, 6]


data = preprocess(data, to_ignore)
# Build neural network
Adam = tflearn.Adam(learning_rate=0.00001, beta1=0.99,name="Adam")
net1 = tflearn.input_data(shape=[None, 6],name="v1")
net2 = tflearn.fully_connected(net1, 32,name="v2")
net3 = tflearn.fully_connected(net2, 32)
net4 = tflearn.fully_connected(net3, 2, activation='softmax')
net5 = tflearn.regression(net4)
# Define model
model = tflearn.DNN(net5)
# Start training (apply gradient descent algorithm)
model.fit(data, labels, n_epoch=10, batch_size=16, show_metric=True)


# Let's create some data for DiCaprio and Winslet
dicaprio = [3, 'Jack Dawson', 'male', 19, 0, 0, 'N/A', 5.0000]
winslet = [1, 'Rose DeWitt Bukater', 'female', 17, 1, 2, 'N/A', 100.0000]
# Preprocess data
dicaprio, winslet = preprocess([dicaprio, winslet], to_ignore)
# Predict surviving chances (class 1 results)
pred = model.predict([dicaprio, winslet])
print("DiCaprio Surviving Rate:", pred[0][1])
print("Winslet Surviving Rate:", pred[1][1])

model.save ("/home/haijunz/tensor-src/zhj/tf-model/zhjmodel_tflean")

print("model  saving..ss33.")


Training Step: 819  | total loss: 0.48967 | time: 0.288s
| Adam | epoch: 010 | loss: 0.48967 - acc: 0.7878 -- iter: 1296/1309
Training Step: 820  | total loss: 0.47422 | time: 0.291s
| Adam | epoch: 010 | loss: 0.47422 - acc: 0.8090 -- iter: 1309/1309
--
('DiCaprio Surviving Rate:', 0.13264095783233643)
('Winslet Surviving Rate:', 0.9119688868522644)
INFO:tensorflow:/home/haijunz/tensor-src/zhj/tf-model/zhjmodel_tflean is not in all_model_checkpoint_paths. Manually adding it.
model  saving..ss33.


In [1]:
from __future__ import absolute_import, division, print_function
import tflearn
import tensorflow as tf
import numpy as np

input_layer = tflearn.input_data(shape=[None, 784], name='input')
dense1 = tflearn.fully_connected(input_layer, 128, name='dense1')
dense2 = tflearn.fully_connected(dense1, 256, name='dense2')
softmax = tflearn.fully_connected(dense2, 10, activation='softmax',name='softmax')
regression = tflearn.regression(softmax, optimizer='adam', learning_rate=0.001,
                                loss='categorical_crossentropy')

model = tflearn.DNN(regression)
X = np.zeros(shape=(1, 784))
Y = np.zeros(shape=(1, 10))
model.fit(X, Y, n_epoch=1, show_metric=True, snapshot_epoch=False)

ops = tf.get_collection_ref(tf.GraphKeys.TRAIN_OPS)  # type: list
for op in ops:
    ops.remove(op)
model.save('/tmp/tflearn_logs/model1.tfl')

Training Step: 1  | time: 0.093s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 1/1
INFO:tensorflow:/tmp/tflearn_logs/model1.tfl is not in all_model_checkpoint_paths. Manually adding it.


In [1]:
""" An example showing how to save/restore models and retrieve weights. """

from __future__ import absolute_import, division, print_function

import tflearn
import tensorflow as tf

import tflearn.datasets.mnist as mnist

# MNIST Data
X, Y, testX, testY = mnist.load_data(one_hot=True)




# Model
input_layer = tflearn.input_data(shape=[None, 784], name='input')
dense1 = tflearn.fully_connected(input_layer, 128, name='dense1')
dense2 = tflearn.fully_connected(dense1, 256, name='dense2')
softmax = tflearn.fully_connected(dense2, 10, activation='softmax',name='softmax')
regression = tflearn.regression(softmax, optimizer='adam',
                                learning_rate=0.001,
                                loss='categorical_crossentropy')

# Define classifier, with model checkpoint (autosave)
#model = tflearn.DNN(regression, checkpoint_path='model.tfl.ckpt')
model = tflearn.DNN(regression, checkpoint_path='model.tfl.ckpt',tensorboard_verbose=3)






# Train model, with model checkpoint every epoch and every 200 training steps.
model.fit(X, Y, n_epoch=1,
          validation_set=(testX, testY),
          show_metric=True,
          snapshot_epoch=True, # Snapshot (save & evaluate) model every epoch.
          snapshot_step=500, # Snapshot (save & evalaute) model every 500 steps.
          run_id='model_and_weights')


# ---------------------
# Save and load a model
# ---------------------
ops = tf.get_collection_ref(tf.GraphKeys.TRAIN_OPS)  # type: list
for op in ops:
    ops.remove(op)
# Manually save model
model.save("/tmp/tflearn_logs/model.tfl")

# Load a model
model.load("/tmp/tflearn_logs/model.tfl")

# Or Load a model from auto-generated checkpoint
# >> model.load("model.tfl.ckpt-500")

# Resume training
model.fit(X, Y, n_epoch=1,
          validation_set=(testX, testY),
          show_metric=True,
          snapshot_epoch=True,
          run_id='model_and_weights')


# ------------------
# Retrieving weights
# ------------------

# Retrieve a layer weights, by layer name:
dense1_vars = tflearn.variables.get_layer_variables_by_name('dense1')
# Get a variable's value, using model `get_weights` method:
print("Dense1 layer weights:")
print(model.get_weights(dense1_vars[0]))
# Or using generic tflearn function:
print("Dense1 layer biases:")
with model.session.as_default():
    print(tflearn.variables.get_value(dense1_vars[1]))

# It is also possible to retrieve a layer weights through its attributes `W`
# and `b` (if available).
# Get variable's value, using model `get_weights` method:
print("Dense2 layer weights:")
print(model.get_weights(dense2.W))
# Or using generic tflearn function:
print("Dense2 layer biases:")
with model.session.as_default():
    print(tflearn.variables.get_value(dense2.b))

Training Step: 1719  | total loss: 0.36273 | time: 55.597s
| Adam | epoch: 002 | loss: 0.36273 - acc: 0.9039 -- iter: 54976/55000
Training Step: 1720  | total loss: 0.35141 | time: 56.887s
| Adam | epoch: 002 | loss: 0.35141 - acc: 0.9057 | val_loss: 0.31456 - val_acc: 0.9142 -- iter: 55000/55000
--
INFO:tensorflow:/home/haijunz/tensor-src/zhj/model.tfl.ckpt-1720 is not in all_model_checkpoint_paths. Manually adding it.
Dense1 layer weights:
[[ 0.00260477  0.00589918 -0.00739277 ..., -0.00420203  0.00218264
  -0.01234564]
 [-0.01247121 -0.01702414  0.01382977 ...,  0.0010445   0.00640426
   0.00831161]
 [ 0.0126583   0.02141601 -0.02218018 ..., -0.01822089  0.01026019
  -0.00774519]
 ..., 
 [ 0.02864939  0.01492578  0.01030963 ..., -0.00540991  0.00403144
   0.01024558]
 [-0.00432761  0.0073133  -0.00682526 ..., -0.00604162  0.00715768
   0.0119549 ]
 [ 0.02559429  0.00131289 -0.01171902 ..., -0.00147637  0.01923358
  -0.00856409]]
Dense1 layer biases:
[ -1.73819408e-01   1.22581773e-0

https://github.com/tflearn/tflearn/blob/master/examples/basics/weights_persistence.py
Retrieving a layer variables can either be done using the layer name, or directly by using 'W' or 'b' attributes that are supercharged to the layer's returned Tensor.

# https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_mnist.py


with name scope




haijunz@haijunz-ThinkPad-T420:/tmp$ tree tflearn_logs
tflearn_logs
├── 4GGNOL
│   └── events.out.tfevents.1488592726.haijunz-ThinkPad-T420
├── 8U95TL
│   └── events.out.tfevents.1488588267.haijunz-ThinkPad-T420
├── AU2V3U
│   └── events.out.tfevents.1488590463.haijunz-ThinkPad-T420
├── convnet_mnist
│   ├── events.out.tfevents.1488592890.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488592939.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488594534.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488594575.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488594782.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488598277.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488601312.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488601368.haijunz-ThinkPad-T420
│   ├── events.out.tfevents.1488601457.haijunz-ThinkPad-T420
│   └── events.out.tfevents.1488601501.haijunz-ThinkPad-T420
├── G01LA5
│   └── events.out.tfevents.1488598238.haijunz-ThinkPad-T420
├── H2MU13
│   └── events.out.tfevents.1488592216.haijunz-ThinkPad-T420
├── J1103M
│   └── events.out.tfevents.1488594463.haijunz-ThinkPad-T420
├── W2GXLT
│   └── events.out.tfevents.1488590347.haijunz-ThinkPad-T420
├── X4DGVP
│   └── events.out.tfevents.1488598162.haijunz-ThinkPad-T420
└── Z4980E
    └── events.out.tfevents.1488590423.haijunz-ThinkPad-T420

10 directories, 19 files
haijunz@haijunz-ThinkPad-T420:/tmp$  tensorboard --logdir=/tmp/tflearn_logs




Starting TensorBoard 41 on port 6006
(You can navigate to http://127.0.1.1:6006)



http://tflearn.org/helpers/trainer/

Trainer

tflearn.helpers.trainer.Trainer (train_ops, graph=None, clip_gradients=5.0, tensorboard_dir='/tmp/tflearn_logs/', tensorboard_verbose=0, checkpoint_path=None, best_checkpoint_path=None, max_checkpoints=None, keep_checkpoint_every_n_hours=10000.0, random_seed=None, session=None, best_val_accuracy=0.0)

http://tflearn.org/getting_started/#training-evaluating-predicting


Training, Evaluating & Predicting

Training functions are another core feature of TFLearn. In Tensorflow, there are no pre-built API to train a network, so TFLearn integrates a set of functions that can easily handle any neural network training, whatever the number of inputs, outputs and optimizers.

While using TFlearn layers, many parameters are already self managed, so it is very easy to train a model, using DNN model class:

network = ... (some layers) ...
network = regression(network, optimizer='sgd', loss='categorical_crossentropy')

model = DNN(network)
model.fit(X, Y)
It can also directly be called for prediction, or evaluation:

network = ...

model = DNN(network)
model.load('model.tflearn')
model.predict(X)
To learn more about these wrappers, see: dnn and estimator.
Visualization


Visualization

While writing a Tensorflow model and adding tensorboard summaries isn't very practical, TFLearn has the ability to self managed a lot of useful logs. Currently, TFLearn supports a verbose level to automatically manage summaries:

0: Loss & Metric (Best speed).
1: Loss, Metric & Gradients.
2: Loss, Metric, Gradients & Weights.
3: Loss, Metric, Gradients, Weights, Activations & Sparsity (Best Visualization).
Using DNN model class, it just requires to specify the verbose argument:

model = DNN(network, tensorboard_verbose=3)
Then, Tensorboard can be run to visualize network and performance:

$ tensorboard --logdir='/tmp/tflearn_logs'


In [2]:
# -*- coding: utf-8 -*-

""" Convolutional Neural Network for MNIST dataset classification task.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
"""

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

AdamX= tflearn.Adam(learning_rate=0.00001, beta1=0.99,name="AdamX")
# Building convolutional network

network = input_data(shape=[None, 28, 28, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer=AdamX, learning_rate=0.01,loss='categorical_crossentropy', name='target')
#worked ...network = regression(network, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='target')
# Training
model = tflearn.DNN(network, tensorboard_verbose=3)

model.save ("/home/haijunz/tensor-src/zhj/tf-model/zhjmodel_tflean1")
model.save ("/tmp/tflearn_logs")
print("model  saving..ss33.")

model.fit({'input': X}, {'target': Y}, n_epoch=1,
           validation_set=({'input': testX}, {'target': testY}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')

Training Step: 859  | total loss: 1.53392 | time: 400.068s
| Adam | epoch: 001 | loss: 1.53392 - acc: 0.6512 -- iter: 54976/55000
Training Step: 860  | total loss: 1.52995 | time: 406.094s
| Adam | epoch: 001 | loss: 1.52995 - acc: 0.6564 | val_loss: 1.48519 - val_acc: 0.6960 -- iter: 55000/55000
--


# tensorflow.train.import_meta_graph does not work?

http://stackoverflow.com/questions/38829641/tensorflow-train-import-meta-graph-does-not-work

To reuse a MetaGraphDef, you will need to record the names of interesting tensors in your original graph. For example, in the first program, set an explicit name argument in the definition of v1, v2 and v4:

v1 = tf.placeholder(tf.float32, name="v1")
v2 = tf.placeholder(tf.float32, name="v2")
# ...
v4 = tf.add(v3, c1, name="v4")
Then, you can use the string names of the tensors in the original graph in your call to sess.run(). For example, the following snippet should work:

import tensorflow as tf
_ = tf.train.import_meta_graph("./file")

sess = tf.Session()
result = sess.run("v4:0", feed_dict={"v1:0": 12.0, "v2:0": 3.3})
Alternatively, you can use tf.get_default_graph().get_tensor_by_name() to get tf.Tensor objects for the tensors of interest, which you can then pass to sess.run():



In [5]:



import tensorflow as tf

tf.reset_default_graph()
v1 = tf.placeholder(tf.float32, name="v1") 
v2 = tf.placeholder(tf.float32, name="v2")
v3 = tf.multiply(v1, v2)
vx = tf.Variable(10.0, name="vx")
v4 = tf.add(v3, vx, name="v4")
saver = tf.train.Saver([vx])
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(vx.assign(tf.add(vx, vx)))
result = sess.run(v4, feed_dict={v1:12.0, v2:3.3})
print(result)
saver.save(sess, "./model_ex1")


_ = tf.train.import_meta_graph("model_ex1.meta")
sess = tf.Session()
result = sess.run("v4:0", feed_dict={"v1:0": 12.0, "v2:0": 10,"vx:0": 3.3})
print("model_ex1================(method 1) ......============")
print(result)

_ = tf.train.import_meta_graph("./model_ex1.meta")
g = tf.get_default_graph()
v1 = g.get_tensor_by_name("v1:0")
v2 = g.get_tensor_by_name("v2:0")
v4 = g.get_tensor_by_name("v4:0")
sess = tf.Session()
result = sess.run(v4, feed_dict={v1: 12.0, v2: 10.0,vx:1})
print(result)
print("model_ex1==========(method 2)==================")
saver = tf.train.import_meta_graph("./model_ex1.meta")

v1_4 = tf.placeholder(tf.float32, name="v1") 
sess = tf.Session()
saver.restore(sess, "./model_ex1")
result = sess.run(v4, feed_dict={v1: 24.0, v2: 10.0,vx:1})

print(result)
print("model_ex1==========(method 3)==================")



Instructions for updating:
Use `tf.global_variables_initializer` instead.
59.6
model_ex1================(method 1) ......============
123.3
121.0
model_ex1==========(method 2)==================
241.0
model_ex1==========(method 3)==================


# Extending TensorFlow

http://tflearn.org/examples/
Layers. Use TFLearn layers along with TensorFlow.
Trainer. Use TFLearn trainer class to train any TensorFlow graph.
Built-in Ops. Use TFLearn built-in operations along with TensorFlow.
Summaries. Use TFLearn summarizers along with TensorFlow.
Variables. Use TFLearn variables along with TensorFlow


In [4]:
#https://github.com/tflearn/tflearn/blob/master/examples/extending_tensorflow/layers.py
"""
This tutorial will introduce how to combine TFLearn and Tensorflow, using
TFLearn trainer with regular Tensorflow graph.
"""
from __future__ import print_function

import tensorflow as tf
import tflearn

# --------------------------------------
# High-Level API: Using TFLearn wrappers
# --------------------------------------

# Using MNIST Dataset
import tflearn.datasets.mnist as mnist
mnist_data = mnist.read_data_sets(one_hot=True)

# User defined placeholders
with tf.Graph().as_default():
    # Placeholders for data and labels
    X = tf.placeholder(shape=(None, 784), dtype=tf.float32)
    Y = tf.placeholder(shape=(None, 10), dtype=tf.float32)

    net = tf.reshape(X, [-1, 28, 28, 1])

    # Using TFLearn wrappers for network building
    net = tflearn.conv_2d(net, 32, 3, activation='relu')
    net = tflearn.max_pool_2d(net, 2)
    net = tflearn.local_response_normalization(net)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.conv_2d(net, 64, 3, activation='relu')
    net = tflearn.max_pool_2d(net, 2)
    net = tflearn.local_response_normalization(net)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 128, activation='tanh')
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 256, activation='tanh')
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 10, activation='linear')

    # Defining other ops using Tensorflow
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net, Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

    # Initializing the variables
    init = tf.initialize_all_variables()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)

        batch_size = 128
        for epoch in range(2): # 2 epochs
            avg_cost = 0.
            total_batch = int(mnist_data.train.num_examples/batch_size)
            for i in range(total_batch):
                batch_xs, batch_ys = mnist_data.train.next_batch(batch_size)
                sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
                cost = sess.run(loss, feed_dict={X: batch_xs, Y: batch_ys})
                avg_cost += cost/total_batch
                if i % 20 == 0:
                    print("Epoch:", '%03d' % (epoch+1), "Step:", '%03d' % i,
                          "Loss:", str(cost))

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


ValueError: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

In [5]:
#https://github.com/tflearn/tflearn/blob/master/examples/extending_tensorflow/trainer.py

"""
This tutorial will introduce how to combine TFLearn and Tensorflow, using
TFLearn wrappers regular Tensorflow expressions.
"""

import tensorflow as tf
import tflearn

# ----------------------------
# Utils: Using TFLearn Trainer
# ----------------------------

# Loading MNIST complete dataset
import tflearn.datasets.mnist as mnist
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

# Define a dnn using Tensorflow
with tf.Graph().as_default():

    # Model variables
    X = tf.placeholder("float", [None, 784])
    Y = tf.placeholder("float", [None, 10])

    W1 = tf.Variable(tf.random_normal([784, 256]))
    W2 = tf.Variable(tf.random_normal([256, 256]))
    W3 = tf.Variable(tf.random_normal([256, 10]))
    b1 = tf.Variable(tf.random_normal([256]))
    b2 = tf.Variable(tf.random_normal([256]))
    b3 = tf.Variable(tf.random_normal([10]))

    # Multilayer perceptron
    def dnn(x):
        x = tf.nn.tanh(tf.add(tf.matmul(x, W1), b1))
        x = tf.nn.tanh(tf.add(tf.matmul(x, W2), b2))
        x = tf.add(tf.matmul(x, W3), b3)
        return x

    net = dnn(X)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net, Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(Y, 1)), tf.float32),
        name='acc')

    # Using TFLearn Trainer
    # Define a training op (op for backprop, only need 1 in this model)
    trainop = tflearn.TrainOp(loss=loss, optimizer=optimizer,
                              metric=accuracy, batch_size=128)

    # Create Trainer, providing all training ops. Tensorboard logs stored
    # in /tmp/tflearn_logs/. It is possible to change verbose level for more
    # details logs about gradients, variables etc...
    trainer = tflearn.Trainer(train_ops=trainop, tensorboard_verbose=0)
    # Training for 10 epochs.
    trainer.fit({X: trainX, Y: trainY}, val_feed_dicts={X: testX, Y: testY},
                n_epoch=10, show_metric=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


ValueError: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

In [6]:
#https://github.com/tflearn/tflearn/blob/master/examples/extending_tensorflow/builtin_ops.py

from __future__ import division, print_function, absolute_import

"""
This tutorial will introduce how to combine TFLearn built-in ops with any
Tensorflow graph.
"""

import tensorflow as tf
import tflearn

# ----------------------------------
# Using TFLearn built-in ops example
# ----------------------------------

# Using MNIST Dataset
import tflearn.datasets.mnist as mnist
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

# User defined placeholders
with tf.Graph().as_default():

    # Model variables
    X = tf.placeholder("float", [None, 784])
    Y = tf.placeholder("float", [None, 10])

    W1 = tf.Variable(tf.random_normal([784, 256]))
    W2 = tf.Variable(tf.random_normal([256, 256]))
    W3 = tf.Variable(tf.random_normal([256, 10]))
    b1 = tf.Variable(tf.random_normal([256]))
    b2 = tf.Variable(tf.random_normal([256]))
    b3 = tf.Variable(tf.random_normal([10]))

    # Multilayer perceptron
    def dnn(x):
        # Using TFLearn PReLU activations ops
        x = tflearn.prelu(tf.add(tf.matmul(x, W1), b1))
        tflearn.summaries.monitor_activation(x) # Monitor activation
        x = tflearn.prelu(tf.add(tf.matmul(x, W2), b2))
        tflearn.summaries.monitor_activation(x) # Monitor activation
        x = tf.nn.softmax(tf.add(tf.matmul(x, W3), b3))
        return x

    net = dnn(X)

    # Using objective ops from TFLearn to compute crossentropy
    loss = tflearn.categorical_crossentropy(net, Y)

    # Using metric ops from TFLearn to compute accuracy
    acc = tflearn.metrics.accuracy_op(net, Y)

    # Using TFLearn SGD Optimizer class
    optimizer = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=200)
    # Because of lr decay, it is required to first build the Optimizer with
    # the step tensor that will monitor training step.
    # (Note: When using TFLearn estimators wrapper, build is self managed,
    # so only using above `Optimizer` class as `DNN` optimizer arg is enough).
    step = tflearn.variable("step", initializer='zeros', shape=[])
    optimizer.build(step_tensor=step)
    optim_tensor = optimizer.get_tensor()

    # Using TFLearn Trainer
    # Define a training op (op for backprop, only need 1 in this model)
    trainop = tflearn.TrainOp(loss=loss, optimizer=optim_tensor,
                              metric=acc, batch_size=128,
                              step_tensor=step)

    # Create Trainer, providing all training ops. Tensorboard logs stored
    # in /tmp/tflearn_logs/. It is possible to change verbose level for more
    # details logs about gradients, variables etc...
    trainer = tflearn.Trainer(train_ops=trainop, tensorboard_verbose=0)
    # Training for 10 epochs.
    trainer.fit({X: trainX, Y: trainY}, val_feed_dicts={X: testX, Y: testY},
                n_epoch=10, show_metric=True)

Training Step: 4299  | total loss: 10.23656 | time: 10.437s
| Optimizer | epoch: 010 | loss: 10.23656 - Accuracy/Mean: 0.5553 -- iter: 54912/55000
Training Step: 4300  | total loss: 10.36420 | time: 11.621s
| Optimizer | epoch: 010 | loss: 10.36420 - Accuracy/Mean: 0.5498 | val_loss: 10.20576 - val_acc: 0.5558 -- iter: 55000/55000
--


In [7]:
#https://github.com/tflearn/tflearn/blob/master/examples/extending_tensorflow/summaries.py

"""
This example introduces the use of TFLearn functions to easily summarize
variables into tensorboard.
TFLearn can summarize:
- Loss / Accuracy: The model loss and accuracy over training steps.
- Activations: Histogram of operation output values.(Requires to add each
    activation to monitor into tf.Graphkeys.ACTIVATIONS collection).
- Gradients: Histogram of trainable variables gradient.
- Weights: Histogram of trainable variables weights.
- Weight Decay: Decay of trainable variables with regularizer. (Requires
    to add each decay into tf.Graphkeys.REGULARIZATION_LOSSES collection)
- Sparsity: Sparsity of trainable variables.
It is useful to also be able to periodically monitor various variables
during training, e.g. confusion matrix entries or AUC metrics. This
can be done using "validation_monitors", an argument to regression or
TrainOp; this argument takes a list of Tensor variables, and passes
them to the trainer, where they are evaluated each time a validation
step happens. The evaluation results are then summarized, and saved
for tensorboard visualization.
Summaries are monitored according to the following verbose levels:
- 0: Loss & Metric (Best speed).
- 1: Loss, Metric & Gradients.
- 2: Loss, Metric, Gradients & Weights.
- 3: Loss, Metric, Gradients, Weights, Activations & Sparsity (Best
     Visualization).
Note: If you are using TFLearn layers, summaries are automatically handled,
so you do not need to manually add them.
"""

import tensorflow as tf
import tflearn

# Loading MNIST dataset
import tflearn.datasets.mnist as mnist
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

# Define a dnn using Tensorflow
with tf.Graph().as_default():

    # Model variables
    X = tf.placeholder("float", [None, 784])
    Y = tf.placeholder("float", [None, 10])

    # Multilayer perceptron, with `tanh` functions activation monitor
    def dnn(x):
        with tf.name_scope('Layer1'):
            W1 = tf.Variable(tf.random_normal([784, 256]), name="W1")
            b1 = tf.Variable(tf.random_normal([256]), name="b1")
            x = tf.nn.tanh(tf.add(tf.matmul(x, W1), b1))
            # Add this `tanh` op to activations collection or monitoring
            tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, x)
            # Add weights regularizer (Regul. summary automatically added)
            tflearn.add_weights_regularizer(W1, 'L2', weight_decay=0.001)

        with tf.name_scope('Layer2'):
            W2 = tf.Variable(tf.random_normal([256, 256]), name="W2")
            b2 = tf.Variable(tf.random_normal([256]), name="b2")
            x = tf.nn.tanh(tf.add(tf.matmul(x, W2), b2))
            # Add this `tanh` op to activations collection or monitoring
            tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, x)
            # Add weights regularizer (Regul. summary automatically added)
            tflearn.add_weights_regularizer(W2, 'L2', weight_decay=0.001)

        with tf.name_scope('Layer3'):
            W3 = tf.Variable(tf.random_normal([256, 10]), name="W3")
            b3 = tf.Variable(tf.random_normal([10]), name="b3")
            x = tf.add(tf.matmul(x, W3), b3)

        return x

    net = dnn(X)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net, Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(Y, 1)), tf.float32),
        name="acc")

    # construct two varaibles to add as additional "valiation monitors"
    # these varaibles are evaluated each time validation happens (eg at a snapshot)
    # and the results are summarized and output to the tensorboard events file,
    # together with the accuracy and loss plots.
    #
    # Here, we generate a dummy variable given by the sum over the current
    # network tensor, and a constant variable.  In practice, the validation
    # monitor may present useful information, like confusion matrix
    # entries, or an AUC metric.
    with tf.name_scope('CustomMonitor'):
        test_var = tf.reduce_sum(tf.cast(net, tf.float32), name="test_var")
        test_const = tf.constant(32.0, name="custom_constant")

    # Define a train op
    trainop = tflearn.TrainOp(loss=loss, optimizer=optimizer,
                              validation_monitors=[test_var, test_const],
                              metric=accuracy, batch_size=128)

    # Tensorboard logs stored in /tmp/tflearn_logs/. Using verbose level 2.
    trainer = tflearn.Trainer(train_ops=trainop,
                              tensorboard_dir='/tmp/tflearn_logs/',
                              tensorboard_verbose=2)
    # Training for 10 epochs.
    trainer.fit({X: trainX, Y: trainY}, val_feed_dicts={X: testX, Y: testY},
                n_epoch=10, show_metric=True, run_id='Summaries_example')

    # Run the following command to start tensorboard:
    # >> tensorboard /tmp/tflearn_logs/
    # Navigate with your web browser to http://0.0.0.0:6006/

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


ValueError: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

In [8]:
#https://github.com/tflearn/tflearn/blob/master/examples/extending_tensorflow/variables.py

"""
This example introduces the use of TFLearn variables to easily implement
Tensorflow variables with custom initialization and regularization.
Note: If you are using TFLearn layers, inititalization and regularization
are directly defined at the layer definition level and applied to inner
variables.
"""

import tensorflow as tf
import tflearn
import tflearn.variables as va

# Loading MNIST dataset
import tflearn.datasets.mnist as mnist
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

# Define a dnn using Tensorflow
with tf.Graph().as_default():

    # Model variables
    X = tf.placeholder("float", [None, 784])
    Y = tf.placeholder("float", [None, 10])

    # Multilayer perceptron
    def dnn(x):
        with tf.variable_scope('Layer1'):
            # Creating variable using TFLearn
            W1 = va.variable(name='W', shape=[784, 256],
                             initializer='uniform_scaling',
                             regularizer='L2')
            b1 = va.variable(name='b', shape=[256])
            x = tf.nn.tanh(tf.add(tf.matmul(x, W1), b1))

        with tf.variable_scope('Layer2'):
            W2 = va.variable(name='W', shape=[256, 256],
                             initializer='uniform_scaling',
                             regularizer='L2')
            b2 = va.variable(name='b', shape=[256])
            x = tf.nn.tanh(tf.add(tf.matmul(x, W2), b2))

        with tf.variable_scope('Layer3'):
            W3 = va.variable(name='W', shape=[256, 10],
                             initializer='uniform_scaling')
            b3 = va.variable(name='b', shape=[10])
            x = tf.add(tf.matmul(x, W3), b3)

        return x

    net = dnn(X)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net, Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(Y, 1)), tf.float32),
        name='acc')

    # Define a train op
    trainop = tflearn.TrainOp(loss=loss, optimizer=optimizer,
                              metric=accuracy, batch_size=128)

    trainer = tflearn.Trainer(train_ops=trainop, tensorboard_verbose=3,
                              tensorboard_dir='/tmp/tflearn_logs/')
    # Training for 10 epochs.
    trainer.fit({X: trainX, Y: trainY}, val_feed_dicts={X: testX, Y: testY},
                n_epoch=10, show_metric=True, run_id='Variables_example')

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


ValueError: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

# Computer Vision


In [1]:
#https://github.com/tflearn/tflearn/blob/master/examples/images/dnn.py

# -*- coding: utf-8 -*-

""" Deep Neural Network for MNIST dataset classification task.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
"""
from __future__ import division, print_function, absolute_import

import tflearn

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)

# Building deep neural network
input_layer = tflearn.input_data(shape=[None, 784])
dense1 = tflearn.fully_connected(input_layer, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.8)
dense2 = tflearn.fully_connected(dropout1, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.8)
softmax = tflearn.fully_connected(dropout2, 10, activation='softmax')

# Regression using SGD with learning rate decay and Top-3 accuracy
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=1000)
top_k = tflearn.metrics.Top_k(3)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=20, validation_set=(testX, testY),
          show_metric=True, run_id="dense_model")

Training Step: 17199  | total loss: 0.15596 | time: 5.341s
| SGD | epoch: 020 | loss: 0.15596 - top3: 0.9936 -- iter: 54976/55000
Training Step: 17200  | total loss: 0.15071 | time: 6.483s
| SGD | epoch: 020 | loss: 0.15071 - top3: 0.9936 | val_loss: 0.11902 - val_acc: 0.9942 -- iter: 55000/55000
--


# Fine-tuning


In [ ]:
""" Finetuning Example. Using weights from model trained in
convnet_cifar10.py to retrain network for a new task (your own dataset).
All weights are restored except last layer (softmax) that will be retrained
to match the new task (finetuning).
"""

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

# Data loading
# Note: You input here any dataset you would like to finetune
X, Y = your_dataset()
num_classes = 10

# Redefinition of convnet_cifar10 network
network = input_data(shape=[None, 32, 32, 3])
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.75)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.5)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
# Finetuning Softmax layer (Setting restore=False to not restore its weights)
softmax = fully_connected(network, num_classes, activation='softmax', restore=False)
regression = regression(softmax, optimizer='adam',
                        loss='categorical_crossentropy',
                        learning_rate=0.001)

model = tflearn.DNN(regression, checkpoint_path='model_finetuning',
                    max_checkpoints=3, tensorboard_verbose=0)
# Load pre-existing model, restoring all weights, except softmax layer ones
model.load('cifar10_cnn')

# Start finetuning
model.fit(X, Y, n_epoch=10, validation_set=0.1, shuffle=True,
          show_metric=True, batch_size=64, snapshot_step=200,
          snapshot_epoch=False, run_id='model_finetuning')

model.save('model_finetuning')

https://github.com/tflearn/tflearn/blob/master/examples/basics/linear_regression.py
    

In [2]:
""" Linear Regression Example """

from __future__ import absolute_import, division, print_function

import tflearn

# Regression data
X = [3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,7.042,10.791,5.313,7.997,5.654,9.27,3.1]
Y = [1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,2.827,3.465,1.65,2.904,2.42,2.94,1.3]

# Linear Regression graph
input_ = tflearn.input_data(shape=[None])
linear = tflearn.single_unit(input_)
regression = tflearn.regression(linear, optimizer='sgd', loss='mean_square',
                                metric='R2', learning_rate=0.01)
m = tflearn.DNN(regression)
m.fit(X, Y, n_epoch=1000, show_metric=True, snapshot_epoch=False)

print("\nRegression result:")
print("Y = " + str(m.get_weights(linear.W)) +
      "*X + " + str(m.get_weights(linear.b)))

print("\nTest prediction for x = 3.2, 3.3, 3.4:")
print(m.predict([3.2, 3.3, 3.4]))
# should output (close, not exact) y = [1.5315033197402954, 1.5585315227508545, 1.5855598449707031]

Training Step: 1000  | total loss: 0.15719 | time: 0.003s
| SGD | epoch: 1000 | loss: 0.15719 - R2: 0.9803 -- iter: 17/17

Regression result:
Y = [ 0.22911617]*X + [ 0.95844913]

Test prediction for x = 3.2, 3.3, 3.4:
[1.6916208267211914, 1.714532494544983, 1.7374441623687744]


https://github.com/tflearn/tflearn/blob/master/examples/basics/logical.py

In [3]:
# -*- coding: utf-8 -*-
"""
Simple Example to train logical operators
"""

from __future__ import absolute_import, division, print_function

import tensorflow as tf
import tflearn

# Logical NOT operator
X = [[0.], [1.]]
Y = [[1.], [0.]]

# Graph definition
with tf.Graph().as_default():
    g = tflearn.input_data(shape=[None, 1])
    g = tflearn.fully_connected(g, 128, activation='linear')
    g = tflearn.fully_connected(g, 128, activation='linear')
    g = tflearn.fully_connected(g, 1, activation='sigmoid')
    g = tflearn.regression(g, optimizer='sgd', learning_rate=2.,
                           loss='mean_square')

    # Model training
    m = tflearn.DNN(g)
    m.fit(X, Y, n_epoch=100, snapshot_epoch=False)

    # Test model
    print("Testing NOT operator")
    print("NOT 0:", m.predict([[0.]]))
    print("NOT 1:", m.predict([[1.]]))

# Logical OR operator
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
Y = [[0.], [1.], [1.], [1.]]

# Graph definition
with tf.Graph().as_default():
    g = tflearn.input_data(shape=[None, 2])
    g = tflearn.fully_connected(g, 128, activation='linear')
    g = tflearn.fully_connected(g, 128, activation='linear')
    g = tflearn.fully_connected(g, 1, activation='sigmoid')
    g = tflearn.regression(g, optimizer='sgd', learning_rate=2.,
                           loss='mean_square')

    # Model training
    m = tflearn.DNN(g)
    m.fit(X, Y, n_epoch=100, snapshot_epoch=False)

    # Test model
    print("Testing OR operator")
    print("0 or 0:", m.predict([[0., 0.]]))
    print("0 or 1:", m.predict([[0., 1.]]))
    print("1 or 0:", m.predict([[1., 0.]]))
    print("1 or 1:", m.predict([[1., 1.]]))

# Logical AND operator
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
Y = [[0.], [0.], [0.], [1.]]

# Graph definition
with tf.Graph().as_default():
    g = tflearn.input_data(shape=[None, 2])
    g = tflearn.fully_connected(g, 128, activation='linear')
    g = tflearn.fully_connected(g, 128, activation='linear')
    g = tflearn.fully_connected(g, 1, activation='sigmoid')
    g = tflearn.regression(g, optimizer='sgd', learning_rate=2.,
                           loss='mean_square')

    # Model training
    m = tflearn.DNN(g)
    m.fit(X, Y, n_epoch=100, snapshot_epoch=False)

    # Test model
    print("Testing AND operator")
    print("0 and 0:", m.predict([[0., 0.]]))
    print("0 and 1:", m.predict([[0., 1.]]))
    print("1 and 0:", m.predict([[1., 0.]]))
    print("1 and 1:", m.predict([[1., 1.]]))

'''
Going further: Graph combination with multiple optimizers
Create a XOR operator using product of NAND and OR operators
'''
# Data
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
Y_nand = [[1.], [1.], [1.], [0.]]
Y_or = [[0.], [1.], [1.], [1.]]

# Graph definition
with tf.Graph().as_default():
    # Building a network with 2 optimizers
    g = tflearn.input_data(shape=[None, 2])
    # Nand operator definition
    g_nand = tflearn.fully_connected(g, 32, activation='linear')
    g_nand = tflearn.fully_connected(g_nand, 32, activation='linear')
    g_nand = tflearn.fully_connected(g_nand, 1, activation='sigmoid')
    g_nand = tflearn.regression(g_nand, optimizer='sgd',
                                learning_rate=2.,
                                loss='binary_crossentropy')
    # Or operator definition
    g_or = tflearn.fully_connected(g, 32, activation='linear')
    g_or = tflearn.fully_connected(g_or, 32, activation='linear')
    g_or = tflearn.fully_connected(g_or, 1, activation='sigmoid')
    g_or = tflearn.regression(g_or, optimizer='sgd',
                              learning_rate=2.,
                              loss='binary_crossentropy')
    # XOR merging Nand and Or operators
    g_xor = tflearn.merge([g_nand, g_or], mode='elemwise_mul')

    # Training
    m = tflearn.DNN(g_xor)
    m.fit(X, [Y_nand, Y_or], n_epoch=400, snapshot_epoch=False)

    # Testing
    print("Testing XOR operator")
    print("0 xor 0:", m.predict([[0., 0.]]))
    print("0 xor 1:", m.predict([[0., 1.]]))
    print("1 xor 0:", m.predict([[1., 0.]]))
    print("1 xor 1:", m.predict([[1., 1.]]))

Training Step: 400  | total loss: 0.90185 | time: 0.005s
| SGD_0 | epoch: 400 | loss: 0.44085 -- iter: 4/4
| SGD_1 | epoch: 400 | loss: 0.46100 -- iter: 4/4
Testing XOR operator
0 xor 0: [[0.0011839382350444794]]
0 xor 1: [[0.9974536895751953]]
1 xor 0: [[0.9972860813140869]]
1 xor 1: [[0.0013692948268726468]]
